In [9]:
### all three exchange api works under SGP vpn
import requests
import pandas as pd
import sqlite3
from binance.client import Client

In [2]:
base_url = "https://api.binance.com"
path = "/api/v3/ticker/price"
r = requests.get(base_url + path)
r.json()

[{'symbol': 'ETHBTC', 'price': '0.02805000'},
 {'symbol': 'LTCBTC', 'price': '0.00129400'},
 {'symbol': 'BNBBTC', 'price': '0.00704800'},
 {'symbol': 'NEOBTC', 'price': '0.00010800'},
 {'symbol': 'QTUMETH', 'price': '0.00106400'},
 {'symbol': 'EOSETH', 'price': '0.00022770'},
 {'symbol': 'SNTETH', 'price': '0.00001065'},
 {'symbol': 'BNTETH', 'price': '0.00020260'},
 {'symbol': 'BCCBTC', 'price': '0.00000000'},
 {'symbol': 'GASBTC', 'price': '0.00003940'},
 {'symbol': 'BNBETH', 'price': '0.25110000'},
 {'symbol': 'BTCUSDT', 'price': '88011.80000000'},
 {'symbol': 'ETHUSDT', 'price': '2468.35000000'},
 {'symbol': 'HSRBTC', 'price': '0.00000000'},
 {'symbol': 'OAXETH', 'price': '0.00000000'},
 {'symbol': 'DNTETH', 'price': '0.00000000'},
 {'symbol': 'MCOETH', 'price': '0.00000000'},
 {'symbol': 'ICNETH', 'price': '0.00000000'},
 {'symbol': 'MCOBTC', 'price': '0.00000000'},
 {'symbol': 'WTCBTC', 'price': '0.00000024'},
 {'symbol': 'WTCETH', 'price': '0.00000000'},
 {'symbol': 'LRCBTC', 'p

In [4]:
df = pd.DataFrame(r.json())

In [21]:
symbols = list(df[df['symbol'].str.contains("USDT|USDC")]['symbol'])
len(symbols)

740

In [10]:
# switch to SGP region for codes below
client = Client()

In [11]:
conn = sqlite3.connect("data/cryptos.db") # after execution, cryptos.db is created

In [26]:
def transform_df(df):
    if df.empty:
        raise ValueError("Received empty DataFrame from API call")

    df = df.iloc[:,:6]
    df.columns = ['time','open','high','low','close','volume']
    df['time'] = pd.to_datetime(df['time'], unit='ms', utc=True)
    df.set_index('time', inplace=True)
    df.index = df.index.tz_convert('America/New_York')

    return df

def crypto_sql_handler(symbol, start = "30 min ago"):
    try: # if table exists
        max_date = pd.read_sql(f"select max(time) from {symbol}", conn).values[0][0] # getting max date
        print(max_date)

        df_new = pd.DataFrame(client.get_historical_klines(symbol=symbol,
                                          interval=Client.KLINE_INTERVAL_1MINUTE,
                                          start_str=str(max_date)))
        
        if df_new.empty:
            print(f"No new data received for {symbol}")
            return None

        df_new = transform_df(df_new)
        df_new = df_new[df_new.index>pd.to_datetime(max_date)]

        # only store data without the last row because incomplete
        df_new[:-1].to_sql(symbol, conn, if_exists='append')
        print(len(df_new[:-1]), " new rows found and inserted")

    except: # if no such table
        new_data = pd.DataFrame(client.get_historical_klines(symbol=symbol,
                                          interval=Client.KLINE_INTERVAL_1MINUTE,
                                          start_str = start))
        
        if new_data.empty:
            print(f"No new data received for {symbol}")
            return None 

        new_data = transform_df(new_data)

        # only store data without the last row because incomplete
        new_data[:-1].to_sql(symbol, conn)
        print(f"New data created for {symbol}, imported {len(new_data[:-1])} rows")

In [27]:
for symbol in symbols:
    crypto_sql_handler(symbol)

2025-02-25 15:17:00-05:00
9  new rows found and inserted
2025-02-25 15:17:00-05:00
9  new rows found and inserted
2025-02-25 15:17:00-05:00
9  new rows found and inserted
No new data received for BCCUSDT
New data created for NEOUSDT, imported 29 rows
New data created for LTCUSDT, imported 29 rows
New data created for QTUMUSDT, imported 29 rows
New data created for ADAUSDT, imported 29 rows
New data created for XRPUSDT, imported 29 rows
New data created for EOSUSDT, imported 29 rows
New data created for TUSDUSDT, imported 29 rows
New data created for IOTAUSDT, imported 29 rows
New data created for XLMUSDT, imported 29 rows
New data created for ONTUSDT, imported 29 rows
New data created for TRXUSDT, imported 29 rows
New data created for ETCUSDT, imported 29 rows
New data created for ICXUSDT, imported 29 rows
No new data received for VENUSDT
New data created for NULSUSDT, imported 29 rows
New data created for VETUSDT, imported 29 rows
No new data received for PAXUSDT
No new data received 